In [1]:
TEST = False
name = 'MRL_ellipse_l'

In [2]:
import logging
import os
import sys
import time

import numpy as np
import matplotlib.pyplot as plt
import pykeen
from pykeen.kge_models import Region

import json

In [3]:
%matplotlib inline

In [4]:
logging.basicConfig(level=logging.INFO)
logging.getLogger('pykeen').setLevel(logging.INFO)

In [5]:
print(sys.version)

3.6.8 |Anaconda, Inc.| (default, Feb 21 2019, 18:30:04) [MSC v.1916 64 bit (AMD64)]


In [ ]:
print(time.asctime())

Fri Jul  5 14:52:42 2019


In [ ]:
print(pykeen.get_version())

0.0.26-dev


Check which hyper-parameters are required by Region:

In [ ]:
Region.hyper_params

['embedding_dim',
 'margin_loss',
 'learning_rate',
 'corrupt_relations',
 'normalization_of_entities',
 'init_radius',
 'reg_lambda',
 'loss_type',
 'neg_factor',
 'region_type']

Define output directory:

In [ ]:
today = time.localtime()
output_directory = 'D:/uni/master/models/region_hpo/opt_{}_{}_{}_{}'.format(today.tm_year, today.tm_mon, today.tm_mday, name)
output_directory

'D:/uni/master/models/region_hpo/opt_2019_7_5_MRL_ellipse_lessdim'

Define hyper-parameters inclduing the evaluation information:
 * Define the hyper-parameter optimization mode: **HPO_mode**
 * Define for each hyper-parameter a list of values (single valued lists are possible)
 * Define the number of hyper-parameter optimization iterations: **maximum_number_of_hpo_iters**
 * Define the path to the test set (**test_set_path**) or define the ratio of the training set that should be used as test set (**test_set_ratio**)
 * Define whether you want to compute the metrics (mean rank and hits@k) in raw or in a filtered setting: **filter_negative_triples**

In [ ]:
config = dict(
    metrics                     = ['mean_rank'],
    training_set_path           = 'D:/uni/master/data/FB15K/train.tsv', 
    test_set_path               = 'D:/uni/master/data/FB15K/valid_150.tsv',
    mapping_path                = 'D:/uni/master/data/fb15k/',
    execution_mode              = 'HPO_mode', # Define hyper-parameter optimization mode
    random_seed                 = 5,
    
    kg_embedding_model_name     = 'Region',
    region_type                 = ['ellipse'],
    embedding_dim               = [30],
    normalization_of_entities   = [2],
    init_radius                 = 0,
    reg_lambda                  = [0.01], # [5, 1, 0.1, 0.01, 0],
    loss_type                   = ['MRL'],
    margin_loss                 = [0.5],  # [0.0, 0.02, 0.05, 0.1, 0.2, 0.3],
    
    neg_factor                  = [1],
    corrupt_relations           = [False], # [True, False],
    filter_negative_triples     = False,
    
    es_metric                   = 'custom',
    learning_rate               = [1e-1],
    num_epochs                  = [500] if not TEST else [10],  
    batch_size                  = [2048],
    test_batch_size             = [4096],
    maximum_number_of_hpo_iters = 1, # Define the number HPO iterations
    preferred_device            = 'gpu'
)

config = dict(
    metrics                     = ['mean_rank'],
    training_set_path           = 'D:/uni/master/data/FB15K/train.tsv', 
    test_set_path               = 'D:/uni/master/data/FB15K/valid_150.tsv',
    mapping_path                = 'D:/uni/master/data/fb15k/',
    execution_mode              = 'HPO_mode', # Define hyper-parameter optimization mode
    random_seed                 = 5,
    
    kg_embedding_model_name     = 'Region',
    region_type                 = ['sphere'],
    embedding_dim               = [50],
    normalization_of_entities   = [1,2],
    init_radius                 = 0,
    reg_lambda                  = [100, 10, 1, 0, 0.1, 0.01],
    loss_type                   = ['NLL'],
    margin_loss                 = [0],  # [0.0, 0.02, 0.05, 0.1, 0.2, 0.3],
    
    neg_factor                  = [1, 2, 5],
    corrupt_relations           = [True, False],
    filter_negative_triples     = False,
    
    es_metric                   = 'custom',
    learning_rate               = [1e-1],
    num_epochs                  = [1000] if not TEST else [10],  
    batch_size                  = [2048],
    test_batch_size             = [4096],
    maximum_number_of_hpo_iters = 10, # Define the number HPO iterations
    preferred_device            = 'gpu'
)

Train and evaluate Region:

In [ ]:
results = pykeen.run(
    config=config,
    output_directory=output_directory
)

HPO Iteration:   0%|                                                                             | 0/1 [00:00<?, ?it/s]

{'metrics': 'mean_rank', 'training_set_path': 'D:/uni/master/data/FB15K/train.tsv', 'test_set_path': 'D:/uni/master/data/FB15K/valid_150.tsv', 'mapping_path': 'D:/uni/master/data/fb15k/', 'execution_mode': 'HPO_mode', 'random_seed': 5, 'kg_embedding_model_name': 'Region', 'region_type': 'ellipse', 'embedding_dim': 30, 'normalization_of_entities': 2, 'init_radius': 0, 'reg_lambda': 0.01, 'loss_type': 'MRL', 'margin_loss': 0.5, 'neg_factor': 1, 'corrupt_relations': False, 'filter_negative_triples': False, 'es_metric': 'custom', 'learning_rate': 0.1, 'num_epochs': 500, 'batch_size': 2048, 'test_batch_size': 4096, 'maximum_number_of_hpo_iters': 1, 'preferred_device': 'gpu', 'pykeen-version': '0.0.26-dev', 'num_entities': 14951, 'num_relations': 1345}


INFO:pykeen.utilities.evaluation_utils.metrics_computations:Evaluation took 15.98s seconds


0.12946576553904768 (0.10006450327967042, 0.18333333333333332)


INFO:pykeen.utilities.train_utils:Epoch   0 /  500 (11.0s):  loss: 0.213  val loss: 0.093  value: 0.129  patience:   1
INFO:pykeen.utilities.train_utils:Epoch   1 /  500 (9.6s):  loss: 0.069  val loss: 0.065  value: 0.129  patience:   2
INFO:pykeen.utilities.train_utils:Epoch   2 /  500 (9.5s):  loss: 0.051  val loss: 0.057  value: 0.129  patience:   3
INFO:pykeen.utilities.train_utils:Epoch   3 /  500 (9.5s):  loss: 0.044  val loss: 0.045  value: 0.129  patience:   4
INFO:pykeen.utilities.train_utils:Epoch   4 /  500 (9.6s):  loss: 0.041  val loss: 0.039  value: 0.129  patience:   5
INFO:pykeen.utilities.train_utils:Epoch   5 /  500 (9.7s):  loss: 0.039  val loss: 0.050  value: 0.129  patience:   6
INFO:pykeen.utilities.train_utils:Epoch   6 /  500 (10.0s):  loss: 0.037  val loss: 0.046  value: 0.129  patience:   7
INFO:pykeen.utilities.train_utils:Epoch   7 /  500 (10.6s):  loss: 0.035  val loss: 0.048  value: 0.129  patience:   8
INFO:pykeen.utilities.train_utils:Epoch   8 /  500 (1

...

INFO:pykeen.utilities.train_utils:Epoch 330 /  500 (10.6s):  loss: 0.017  val loss: 0.025  value: 0.239  patience:  11
INFO:pykeen.utilities.train_utils:Epoch 331 /  500 (10.4s):  loss: 0.017  val loss: 0.025  value: 0.239  patience:  12
INFO:pykeen.utilities.train_utils:Epoch 332 /  500 (10.7s):  loss: 0.017  val loss: 0.025  value: 0.239  patience:  13
INFO:pykeen.utilities.train_utils:Epoch 333 /  500 (10.2s):  loss: 0.017  val loss: 0.025  value: 0.239  patience:  14
INFO:pykeen.utilities.train_utils:Epoch 334 /  500 (10.1s):  loss: 0.016  val loss: 0.025  value: 0.239  patience:  15
INFO:pykeen.utilities.train_utils:Epoch 335 /  500 (10.0s):  loss: 0.017  val loss: 0.025  value: 0.239  patience:  16
INFO:pykeen.utilities.train_utils:Epoch 336 /  500 (10.1s):  loss: 0.017  val loss: 0.025  value: 0.239  patience:  17
INFO:pykeen.utilities.train_utils:Epoch 337 /  500 (10.1s):  loss: 0.017  val loss: 0.025  value: 0.239  patience:  18
INFO:pykeen.utilities.train_utils:Epoch 338 /  5

0.23851122919949672 (0.1901014396782807, 0.32)


INFO:pykeen.utilities.train_utils:Epoch 340 /  500 (10.1s):  loss: 0.017  val loss: 0.025  value: 0.239  patience:  21
INFO:pykeen.utilities.train_utils:Epoch 341 /  500 (10.1s):  loss: 0.017  val loss: 0.025  value: 0.239  patience:  22
INFO:pykeen.utilities.train_utils:Epoch 342 /  500 (10.8s):  loss: 0.017  val loss: 0.025  value: 0.239  patience:  23
INFO:pykeen.utilities.train_utils:Epoch 343 /  500 (10.3s):  loss: 0.017  val loss: 0.025  value: 0.239  patience:  24
INFO:pykeen.utilities.train_utils:Epoch 344 /  500 (10.3s):  loss: 0.017  val loss: 0.025  value: 0.239  patience:  25
INFO:pykeen.utilities.train_utils:Epoch 345 /  500 (10.3s):  loss: 0.017  val loss: 0.025  value: 0.239  patience:  26
INFO:pykeen.utilities.train_utils:Epoch 346 /  500 (10.5s):  loss: 0.017  val loss: 0.025  value: 0.239  patience:  27
INFO:pykeen.utilities.train_utils:Epoch 347 /  500 (10.1s):  loss: 0.016  val loss: 0.025  value: 0.239  patience:  28
INFO:pykeen.utilities.train_utils:Epoch 348 /  5

0.23936674428217866 (0.190020284965446, 0.3233333333333333)


INFO:pykeen.utilities.train_utils:Epoch 350 /  500 (10.2s):  loss: 0.017  val loss: 0.025  value: 0.239  patience:  31
INFO:pykeen.utilities.train_utils:Epoch 351 /  500 (10.3s):  loss: 0.017  val loss: 0.025  value: 0.239  patience:  32
INFO:pykeen.utilities.train_utils:Epoch 352 /  500 (10.2s):  loss: 0.017  val loss: 0.025  value: 0.239  patience:  33
INFO:pykeen.utilities.train_utils:Epoch 353 /  500 (10.6s):  loss: 0.017  val loss: 0.025  value: 0.239  patience:  34


In [ ]:
config['reg_lambda'] = [0.1]
results30 = pykeen.run(
    config=config,
    output_directory=output_directory
)

In [ ]:
config['reg_lambda'] = [1]
results40 = pykeen.run(
    config=config,
    output_directory=output_directory
)

In [ ]:
config['reg_lambda'] = [5]
results40 = pykeen.run(
    config=config,
    output_directory=output_directory
)

Check result entries:

In [ ]:
results.results.keys()

In [ ]:
results = {
    'losses': json.load(open(os.path.join(output_directory, 'losses.json'))),
    'validation_losses': json.load(open(os.path.join(output_directory, 'val_losses.json'))),
    'entity_to_embedding': json.load(open(os.path.join(output_directory, 'entities_to_embeddings.json'))),
    'relation_to_embedding': json.load(open(os.path.join(output_directory, 'relations_to_embeddings.json'))),
    'final_configuration': json.load(open(os.path.join(output_directory, 'configuration.json'))),
    'eval_summary': json.load(open(os.path.join(output_directory, 'evaluation_summary.json'))),
    'search_summary': json.load(open(os.path.join(output_directory, 'search_summary.json'))),
    'entity_to_id': json.load(open(os.path.join(output_directory, 'entity_to_id.json'))),
    'relation_to_id': json.load(open(os.path.join(output_directory, 'relation_to_id.json')))
}

In [ ]:
results['search_summary']

In [ ]:
import torch, os
import pykeen.constants as pkc
from pykeen.kge_models import get_kge_model
# Initialize KG embedding model
config = results['final_configuration']
if config['random_seed'] is not None:
    torch.manual_seed(config['random_seed'])
    
model: torch.nn.Module = get_kge_model(config=config)
model.load_state_dict(torch.load(os.path.join(output_directory, 'trained_model.pkl')))

results['trained_model'] = model 

Get the model configuration that lead to best results:

In [ ]:
results['final_configuration']

Access trained model:

In [ ]:
results['trained_model']

Visualize loss values:

In [ ]:
losses = results['losses']
epochs = np.arange(len(losses))
plt.title(r'Loss Per Epoch')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.plot(epochs, losses)
plt.show()

In [ ]:
losses = results['validation_losses']
epochs = np.arange(len(losses))
plt.title(r'Loss Per Epoch')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.plot(epochs, losses)
plt.show()

Get evaluation results:
 * Mean rank
 * Hits@k, k $\in$ {1,3,5,10} 

In [ ]:
results['eval_summary']

1.

    {'training_set_path': 'D:/uni/master/data/YAGO39K/Train/triple2id.tsv',
     'test_set_path': 'D:/uni/master/data/YAGO39K/Valid/instanceOf2id_positive.tsv',
     'neg_test_set_path': 'D:/uni/master/data/YAGO39K/Valid/instanceOf2id_negative.tsv',
     'execution_mode': 'HPO_mode',
     'random_seed': 2,
     'kg_embedding_model_name': 'Region',
     'embedding_dim': 20,
     'normalization_of_entities': 2,
     'init_radius': 5,
     'reg_lambda': 0.02,
     'margin_loss': 0,
     'learning_rate': 5,
     'num_epochs': 30,
     'batch_size': 1024,
     'test_batch_size': 1024,
     'filter_negative_triples': True,
     'maximum_number_of_hpo_iters': 20,
     'preferred_device': 'gpu',
     'pykeen-version': '0.0.26-dev',
     'num_entities': 46100,
     'num_relations': 39}


     {'mean_rank': None,
     'hits@k': None,
     'precision': [0.5817929759704251],
     'recall': [0.7386089563913696],
     'accuracy': [0.6035163257983495],
     'f1_score': 0.6508889335669357}

     30 epochs overfit

2.
   
       {'training_set_path': 'D:/uni/master/data/YAGO39K/Train/instanceOf2id.tsv', 'test_set_path': 'D:/uni/master/data/YAGO39K/Valid/subClassOf2id_positive.tsv', 'neg_test_set_path': 'D:/uni/master/data/YAGO39K/Valid/triple2id_negative.tsv', 'execution_mode': 'HPO_mode', 'random_seed': 2, 'kg_embedding_model_name': 'Region', 'embedding_dim': 100, 'normalization_of_entities': 2, 'init_radius': 0.5, 'reg_lambda': 0.2, 'margin_loss': 0, 'learning_rate': 5, 'num_epochs': 30, 'batch_size': 1024, 'test_batch_size': 1024, 'filter_negative_triples': True, 'maximum_number_of_hpo_iters': 20, 'preferred_device': 'gpu', 'pykeen-version': '0.0.26-dev', 'num_entities': 46100, 'num_relations': 39}

    MetricResults(mean_rank=None, hits_at_k=None, precision=(0.5531790499390986,), recall=(0.7401082067661822,), accuracy=(0.5707994911439476,), fscore=0.6331344448781575)

    18 epochs


3.

    {'training_set_path': 'D:/uni/master/data/YAGO39K/Train/instanceOf2id.tsv', 'test_set_path': 'D:/uni/master/data/YAGO39K/Valid/instanceOf2id_positive.tsv', 'neg_test_set_path': 'D:/uni/master/data/YAGO39K/Valid/triple2id_negative.tsv', 'execution_mode': 'HPO_mode', 'random_seed': 2, 'kg_embedding_model_name': 'Region', 'embedding_dim': 50, 'normalization_of_entities': 2, 'init_radius': 10, 'reg_lambda': 0.1, 'margin_loss': 0, 'learning_rate': 2, 'num_epochs': 30, 'batch_size': 1024, 'test_batch_size': 1024, 'filter_negative_triples': True, 'maximum_number_of_hpo_iters': 20, 'preferred_device': 'gpu', 'pykeen-version': '0.0.26-dev', 'num_entities': 46100, 'num_relations': 39}

    MetricResults(mean_rank=None, hits_at_k=None, precision=(0.5531279471864193,), recall=(0.6881559220389805,), accuracy=(0.5657435495971557,), fscore=0.6132977023847561)

    30 epochs not finished



        {'metrics': 'mean_rank', 'training_set_path': 'D:/uni/master/data/FB15K/train.tsv', 'test_set_path': 'D:/uni/master/data/FB15K/valid_150.tsv', 'mapping_path': 'D:/uni/master/data/fb15k/', 'execution_mode': 'HPO_mode', 'random_seed': 5, 'kg_embedding_model_name': 'Region', 'region_type': 'sphere', 'embedding_dim': 50, 'normalization_of_entities': 2, 'init_radius': 0, 'reg_lambda': 0.01, 'loss_type': 'MRL', 'margin_loss': 0.5, 'neg_factor': 5, 'corrupt_relations': False, 'filter_negative_triples': False, 'es_metric': 'custom', 'learning_rate': 0.1, 'num_epochs': 1000, 'batch_size': 2048, 'test_batch_size': 4096, 'maximum_number_of_hpo_iters': 5, 'preferred_device': 'gpu', 'pykeen-version': '0.0.26-dev', 'num_entities': 14951, 'num_relations': 1345}

### 